In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_digits
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np

# Prepare Data

In [2]:
cifar10_dataset = keras.datasets.cifar10.load_data()
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

X = np.concatenate((x_train, x_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

x_temp, x_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_temp, y_temp, test_size=0.15, random_state=1)

170498071/170498071 [==============================] - 4s 0us/step


In [3]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

(43350, 32, 32, 3)
(9000, 32, 32, 3)
(7650, 32, 32, 3)


In [4]:
x_train, x_test, x_val = x_train/255.0, x_test/255.0,  x_val/255.0

# Define 5 Layer Model

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[32, 32, 3]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(150, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(120, activation="relu"),
    keras.layers.Dense(75, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 150)               15150     
                                                                 
 dense_2 (Dense)             (None, 200)               30200     
                                                                 
 dense_3 (Dense)             (None, 120)               24120     
                                                                 
 dense_4 (Dense)             (None, 75)                9075      
                                                                 
 dense_5 (Dense)             (None, 10)                7

Last Layer Use Softmax

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [8]:
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
1355/1355 [==============================] - 17s 11ms/step - loss: 1.9825 - accuracy: 0.2796 - val_loss: 1.8357 - val_accuracy: 0.3527
Epoch 2/50
1355/1355 [==============================] - 8s 6ms/step - loss: 1.7630 - accuracy: 0.3688 - val_loss: 1.7075 - val_accuracy: 0.3882
Epoch 3/50
1355/1355 [==============================] - 8s 6ms/step - loss: 1.6724 - accuracy: 0.4014 - val_loss: 1.6668 - val_accuracy: 0.4060
Epoch 4/50
1355/1355 [==============================] - 8s 6ms/step - loss: 1.6135 - accuracy: 0.4239 - val_loss: 1.5857 - val_accuracy: 0.4405
Epoch 5/50
1355/1355 [==============================] - 8s 6ms/step - loss: 1.5702 - accuracy: 0.4404 - val_loss: 1.5603 - val_accuracy: 0.4421
Epoch 6/50
1355/1355 [==============================] - 6s 5ms/step - loss: 1.5269 - accuracy: 0.4536 - val_loss: 1.5662 - val_accuracy: 0.4391
Epoch 7/50
1355/1355 [==============================] - 8s 6ms/step - loss: 1.4965 - accuracy: 0.4675 - val_loss: 1.5476 - val_accurac

In [9]:
y_pred = model.predict(x_test)

y_pred_labels = y_pred.argmax(axis=1)

f1 = f1_score(y_test, y_pred_labels, average='weighted')
accuracy = accuracy_score(y_test, y_pred_labels)

print("Weighted F1 Score:", f1)
print("Accuracy:", accuracy)

282/282 [==============================] - 1s 2ms/step
Weighted F1 Score: 0.5109039676808143
Accuracy: 0.5075555555555555


# Optimized Model

In [10]:
!pip install keras-tuner tensorflow

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

import time

start_time = time.time()

# Define a function to build the model
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(32, 32, 3)))

    # Tune the number of dense layers (between 6 and 10 layers)
    num_layers = hp.Int('num_layers', min_value=6, max_value=10)

    for i in range(num_layers):
        # Tune the number of units in each dense layer (between 128 and 256)
        units = hp.Int(f'units_{i}', min_value=128, max_value=256, step=32)
        # Tune the activation function for each dense layer
        activation = hp.Choice(f'activation_{i}', values=['relu', 'tanh'])
        model.add(layers.Dense(units=units, activation=activation))

    model.add(layers.Dense(10, activation='softmax'))

    # Tune the learning rate for the optimizer
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Initialize the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_tuner_dir',
    project_name='cifar10_hyperparameter_tuning'
)

# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the best model
best_model = tuner.hypermodel.build(best_hps)

end_time = time.time()

execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")


Trial 5 Complete [00h 01m 48s]
val_accuracy: 0.3937254846096039

Best val_accuracy So Far: 0.45764705538749695
Total elapsed time: 00h 08m 22s
Execution time: 502.49 seconds


Last Layer activation : Softmax

In [11]:
best_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_8 (Dense)             (None, 128)               393344    
                                                                 
 dense_9 (Dense)             (None, 224)               28896     
                                                                 
 dense_10 (Dense)            (None, 224)               50400     
                                                                 
 dense_11 (Dense)            (None, 256)               57600     
                                                                 
 dense_12 (Dense)            (None, 224)               57568     
                                                                 
 dense_13 (Dense)            (None, 128)              

In [12]:
loss_fn = keras.losses.SparseCategoricalCrossentropy()

best_model.compile(loss=loss_fn,
              optimizer="sgd",
              metrics=["accuracy"])

In [13]:
history = best_model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
1355/1355 [==============================] - 20s 14ms/step - loss: 1.9525 - accuracy: 0.2899 - val_loss: 1.7832 - val_accuracy: 0.3593
Epoch 2/50
1355/1355 [==============================] - 19s 14ms/step - loss: 1.7421 - accuracy: 0.3685 - val_loss: 1.7208 - val_accuracy: 0.3859
Epoch 3/50
1355/1355 [==============================] - 15s 11ms/step - loss: 1.6461 - accuracy: 0.4096 - val_loss: 1.6045 - val_accuracy: 0.4244
Epoch 4/50
1355/1355 [==============================] - 10s 7ms/step - loss: 1.5789 - accuracy: 0.4317 - val_loss: 1.6083 - val_accuracy: 0.4263
Epoch 5/50
1355/1355 [==============================] - 10s 7ms/step - loss: 1.5262 - accuracy: 0.4537 - val_loss: 1.5975 - val_accuracy: 0.4362
Epoch 6/50
1355/1355 [==============================] - 10s 8ms/step - loss: 1.4873 - accuracy: 0.4644 - val_loss: 1.5438 - val_accuracy: 0.4444
Epoch 7/50
1355/1355 [==============================] - 11s 8ms/step - loss: 1.4512 - accuracy: 0.4789 - val_loss: 1.4890 - val

# Data Scores

roc_auc Parameters is for binary classification and does not use in muli class classification.

In [14]:
y_train_pred_labels = best_model.predict(x_train).argmax(axis=1)
y_val_pred_labels = best_model.predict(x_val).argmax(axis=1)
y_test_pred_labels = best_model.predict(x_test).argmax(axis=1)

train_accuracy = accuracy_score(y_train, y_train_pred_labels)
val_accuracy = accuracy_score(y_val, y_val_pred_labels)
test_accuracy = accuracy_score(y_test, y_test_pred_labels)

train_f1 = f1_score(y_train, y_train_pred_labels, average='weighted')
val_f1 = f1_score(y_val, y_val_pred_labels, average='weighted')
test_f1 = f1_score(y_test, y_test_pred_labels, average='weighted')

data = {
    'Dataset': ['Train', 'Validation', 'Test'],
    'Accuracy': [train_accuracy, val_accuracy, test_accuracy],
    'F1 Score': [train_f1, val_f1, test_f1]
}

results_df = pd.DataFrame(data)

table = tabulate(results_df, headers='keys', tablefmt='fancy_grid')

print(table)

282/282 [==============================] - 1s 3ms/step
╒════╤════════════╤════════════╤════════════╕
│    │ Dataset    │   Accuracy │   F1 Score │
╞════╪════════════╪════════════╪════════════╡
│  0 │ Train      │   0.801269 │   0.803477 │
├────┼────────────┼────────────┼────────────┤
│  1 │ Validation │   0.51085  │   0.51222  │
├────┼────────────┼────────────┼────────────┤
│  2 │ Test       │   0.516444 │   0.518744 │
╘════╧════════════╧════════════╧════════════╛


# Wide and Deep Model

In [15]:
def create_wide_and_deep_model():
    input_layer = tf.keras.Input(shape=(32, 32, 3))

    wide = layers.Flatten()(input_layer)
    wide = layers.Dense(128, activation='relu')(wide)
    wide = layers.Dense(64, activation='relu')(wide)

    deep = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
    deep = layers.MaxPooling2D((2, 2))(deep)
    deep = layers.Conv2D(64, (3, 3), activation='relu')(deep)

    deep = layers.Flatten()(deep)

    combined = layers.concatenate([wide, deep])

    combined = layers.Dense(64, activation='relu')(combined)
    output_layer = layers.Dense(10, activation='softmax')(combined)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

wd_model = create_wide_and_deep_model()

wd_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

wd_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 32, 32, 3)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 30, 30, 32)           896       ['input_1[0][0]']             
                                                                                                  
 flatten_2 (Flatten)         (None, 3072)                 0         ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)           0         ['conv2d[0][0]']              
 D)                                                                                           

In [16]:
history = wd_model.fit(x_train, y_train, epochs=10,
                    validation_data=(x_val, y_val))

Epoch 1/10
1355/1355 [==============================] - 70s 51ms/step - loss: 1.4252 - accuracy: 0.4884 - val_loss: 1.1654 - val_accuracy: 0.5855
Epoch 2/10
1355/1355 [==============================] - 68s 50ms/step - loss: 1.0505 - accuracy: 0.6308 - val_loss: 1.0164 - val_accuracy: 0.6473
Epoch 3/10
1355/1355 [==============================] - 71s 52ms/step - loss: 0.9018 - accuracy: 0.6861 - val_loss: 0.9891 - val_accuracy: 0.6569
Epoch 4/10
1355/1355 [==============================] - 67s 49ms/step - loss: 0.7941 - accuracy: 0.7230 - val_loss: 1.0157 - val_accuracy: 0.6458
Epoch 5/10
1355/1355 [==============================] - 67s 49ms/step - loss: 0.6926 - accuracy: 0.7587 - val_loss: 0.9922 - val_accuracy: 0.6659
Epoch 6/10
1355/1355 [==============================] - 68s 50ms/step - loss: 0.6032 - accuracy: 0.7883 - val_loss: 1.0634 - val_accuracy: 0.6579
Epoch 7/10
1355/1355 [==============================] - 65s 48ms/step - loss: 0.5132 - accuracy: 0.8191 - val_loss: 1.0100 -

In [17]:
y_train_pred_labels = wd_model.predict(x_train).argmax(axis=1)
y_val_pred_labels = wd_model.predict(x_val).argmax(axis=1)
y_test_pred_labels = wd_model.predict(x_test).argmax(axis=1)

train_accuracy = accuracy_score(y_train, y_train_pred_labels)
val_accuracy = accuracy_score(y_val, y_val_pred_labels)
test_accuracy = accuracy_score(y_test, y_test_pred_labels)

train_f1 = f1_score(y_train, y_train_pred_labels, average='weighted')
val_f1 = f1_score(y_val, y_val_pred_labels, average='weighted')
test_f1 = f1_score(y_test, y_test_pred_labels, average='weighted')

data = {
    'Dataset': ['Train', 'Validation', 'Test'],
    'Accuracy': [train_accuracy, val_accuracy, test_accuracy],
    'F1 Score': [train_f1, val_f1, test_f1]
}

results_df = pd.DataFrame(data)

table = tabulate(results_df, headers='keys', tablefmt='fancy_grid')

print(table)

282/282 [==============================] - 3s 12ms/step
╒════╤════════════╤════════════╤════════════╕
│    │ Dataset    │   Accuracy │   F1 Score │
╞════╪════════════╪════════════╪════════════╡
│  0 │ Train      │   0.928766 │   0.928694 │
├────┼────────────┼────────────┼────────────┤
│  1 │ Validation │   0.660261 │   0.65956  │
├────┼────────────┼────────────┼────────────┤
│  2 │ Test       │   0.656    │   0.656062 │
╘════╧════════════╧════════════╧════════════╛
